In [ ]:
base_path = "/content/drive/My Drive/Github/mtc-device-activation"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
# Load saved data
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append(f'{base_path}/frameworks')

from cosamp import cosamp

data = np.load('/content/drive/My Drive/Github/mtc-device-activation/data/activity-models/e2-general-model.npz', allow_pickle=True)
all_DeviceLocations = data['all_DeviceLocations']
num_samples = data['num_samples']
device_count = data['device_count']
sparsity = data['sparsity']

data = np.load("/content/drive/My Drive/Github/mtc-device-activation/data/communication-models/e2-communication-simulation-theta.npz", allow_pickle=True)
all_theta = data["all_theta"]

data = np.load("/content/drive/My Drive/Github/mtc-device-activation/data/communication-models/e2-communication-simulation-gamma.npz", allow_pickle=True)
all_gamma = data["all_gamma"]

data = np.load("/content/drive/My Drive/Github/mtc-device-activation/data/communication-models/e2-communication-simulation-z.npz", allow_pickle=True)
all_z = data["all_z"]

data = np.load("/content/drive/My Drive/Github/mtc-device-activation/data/communication-models/e2-communication-simulation-noise.npz", allow_pickle=True)
all_noise = data["all_noise"]

data = np.load("/content/drive/My Drive/Github/mtc-device-activation/data/communication-models/e2-communication-simulation-received-signal.npz", allow_pickle=True)
all_received_signal = data["all_received_signal"]

data = np.load("/content/drive/My Drive/Github/mtc-device-activation/data/communication-models/e2-communication-simulation-general.npz", allow_pickle=True)
snr_db_array = data["snr_db_array"]
pilot_length_array = data["pilot_length_array"]
M = data["M"]

snr_db_index = 2
snr_db=snr_db_array[snr_db_index]

pilot_length_index = 2
pilot_length=pilot_length_array[pilot_length_index]

max_iter = 500

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
sample_range = range(0, 1000)
num_samples = len(sample_range)

In [ ]:
# Placeholder arrays for results
nmse_iter_all = np.zeros((num_samples, max_iter))

In [ ]:
# Loop through each sample with an outer progress bar
for sample_index in tqdm(sample_range, desc="Processing Samples", position=0):
    gamma = all_gamma[sample_index]  # Ground truth gamma values (true support)
    loc = all_DeviceLocations[sample_index]
    theta = all_theta[sample_index, pilot_length_index, :pilot_length]
    received_signal = all_received_signal[sample_index, pilot_length_index, snr_db_index, :pilot_length]
    z = all_z[sample_index]

    # Compute noise
    snr = 10 ** (snr_db / 10)
    signal_power = np.mean(np.abs(np.matmul(theta, z))**2)
    noise_power = signal_power / snr

    # Run CoSaMP
    z_result, z_history, iteration_count = cosamp(
        theta, received_signal, sparsity, max_iter=max_iter
    )

    # NMSE per iteration
    for t, z_est in enumerate(z_history):
        numerator = np.linalg.norm(z[gamma == 1] - z_est[gamma == 1])**2
        denominator = np.linalg.norm(z[gamma == 1])**2
        nmse_iter_all[sample_index, t] = numerator / denominator if denominator != 0 else np.nan

Processing Samples: 100%|██████████| 1000/1000 [10:07<00:00,  1.65it/s]


In [ ]:
np.savez_compressed(
    f'{base_path}/convergence/data/cosamp_snr_db_12_pilot_length_30_convergence.npz',
    nmse_iter_all=nmse_iter_all,
)

print("All results have been saved")

All results have been saved
